In [1]:
# Libraries
import pandas as pd
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords
import re

import pyLDAvis
import pyLDAvis.gensim

In [2]:
# reading in data from contest 853
df = pd.read_csv("853.csv")

df.head()

,caption,mean,precision,votes,not_funny,somewhat_funny,funny
0,Who knew the Swiss had a navy?,1.675661,0.018367,1816,946,513,357
1,Weapons down. It's the Swiss.,1.672884,0.020759,1394,721,408,265
2,"Wow, and that’s just the tip of the Jarlsberg!",1.643323,0.018380,1685,886,512,286
3,All we need now is to find a port,1.639823,0.012828,3837,2151,917,769
4,We must be directly over where Wisconsin used ...,1.638809,0.022989,1041,542,333,166


In [3]:
# removing columns
df_subset = df.drop(columns=['mean', 'precision', 'votes', 'not_funny', 'somewhat_funny', 'funny'], axis=1)

# subsetting first 100 rows
df_subset = df_subset.head(100)

df_subset.head()

,caption
0,Who knew the Swiss had a navy?
1,Weapons down. It's the Swiss.
2,"Wow, and that’s just the tip of the Jarlsberg!"
3,All we need now is to find a port
4,We must be directly over where Wisconsin used ...


In [4]:
# stopwords
stopwords = stopwords.words("english")

In [5]:
# Remove punctuation
df_subset['caption_processed'] = \
df_subset['caption'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
df_subset['caption_processed'] = \
df_subset['caption_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
df_subset.head()

<>:3: DeprecationWarning: invalid escape sequence '\.'
<>:3: DeprecationWarning: invalid escape sequence '\.'
C:\Users\alexc\AppData\Local\Temp\ipykernel_19724\1869826678.py:3: DeprecationWarning: invalid escape sequence '\.'
  df_subset['caption'].map(lambda x: re.sub('[,\.!?]', '', x))


,caption,caption_processed
0,Who knew the Swiss had a navy?,who knew the swiss had a navy
1,Weapons down. It's the Swiss.,weapons down it's the swiss
2,"Wow, and that’s just the tip of the Jarlsberg!",wow and that’s just the tip of the jarlsberg
3,All we need now is to find a port,all we need now is to find a port
4,We must be directly over where Wisconsin used ...,we must be directly over where wisconsin used ...


In [6]:
# lemmatization

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable = ["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(df_subset.caption_processed)
print(lemmatized_texts)

['know swiss navy', 'weapon swiss', '’ just tip', 'need now find port', 'directly wisconsin use', 'aged alright', 'go need big board', '’re go need big board', 'sea muenster', 'pair well great white', 'let hope just tip', 'worry swiss neutral', 'careful man ’re un - charcuterie water', 'perfect great white', 'think ’re go need more cracker', 'look enter costco water', 'careful boy only tip', 'careful just tip', 'gouda', 'careful boy sharp one', 'careful man sharp', 'let pass brie', 'boy go need big board', 'get close', 'call béchamel', 'tell muenster roam sea', 'never say picnic', 'look ’ sharkuterie', 'thar blow muenster deep', 'take cutter', 'wait attack neutral', 'white cheese night dinner delight white cheese morning high cholesterol warning', 'look first take run', 'think go need big cheese board', '’ swiss else see first', 'next time let just go costco', '’re go need big cracker', 'care flee tyranny sandwich island', 'think go need more robust cabernet', 'man island cheese stand 

In [7]:
# removing stopwords and tokenizing
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc = True)
        final.append(new)
    return(final)
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stopwords] for doc in texts]

data_words = gen_words(lemmatized_texts)
data_words = remove_stopwords(data_words)
print(data_words)

[['know', 'swiss', 'navy'], ['weapon', 'swiss'], ['tip'], ['need', 'find', 'port'], ['directly', 'wisconsin', 'use'], ['aged', 'alright'], ['go', 'need', 'big', 'board'], ['go', 'need', 'big', 'board'], ['sea', 'muenster'], ['pair', 'well', 'great', 'white'], ['let', 'hope', 'tip'], ['worry', 'swiss', 'neutral'], ['careful', 'man', 'un', 'charcuterie', 'water'], ['perfect', 'great', 'white'], ['think', 'go', 'need', 'cracker'], ['look', 'enter', 'costco', 'water'], ['careful', 'boy', 'tip'], ['careful', 'tip'], ['gouda'], ['careful', 'boy', 'sharp', 'one'], ['careful', 'man', 'sharp'], ['let', 'pass', 'brie'], ['boy', 'go', 'need', 'big', 'board'], ['get', 'close'], ['call', 'bechamel'], ['tell', 'muenster', 'roam', 'sea'], ['never', 'say', 'picnic'], ['look', 'sharkuterie'], ['thar', 'blow', 'muenster', 'deep'], ['take', 'cutter'], ['wait', 'attack', 'neutral'], ['white', 'cheese', 'night', 'dinner', 'delight', 'white', 'cheese', 'morning', 'high', 'cholesterol', 'warning'], ['look', 

In [8]:
# creating a dictionary

id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
    
print(corpus[0][0:20])

word = id2word[[0][:1][0]]
print(word)

[(0, 1), (1, 1), (2, 1)]
know


In [9]:
# creating model

lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = id2word,
                                           num_topics = 30,
                                           random_state = 100,
                                           update_every = 1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha = "auto")

In [10]:
# visualizing model

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = "mmds", R = 30)

vis

C:\Users\alexc\anaconda3\envs\caption_contest\Lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.289799  0.284694       1        1  10.579926
25     0.284241  0.198167       2        1   8.448445
7     -0.175229  0.300722       3        1   7.157069
23     0.146052  0.229297       4        1   5.335471
28     0.101738 -0.279889       5        1   5.164931
1      0.257984 -0.214446       6        1   4.998095
16     0.000157  0.270153       7        1   4.725785
5     -0.082148 -0.290923       8        1   4.563058
14    -0.263451 -0.024561       9        1   4.258795
26     0.242140  0.030378      10        1   4.155120
18     0.269510 -0.113041      11        1   4.061455
11    -0.082018  0.217761      12        1   3.817993
20    -0.180383 -0.207935      13        1   3.795212
12    -0.305715  0.064949      14        1   3.755766
19     0.060184 -0.206199      15        1   3.680216
10    -0.181593  0.129807      16        1   3.592406
27    -0.286407 -0.086949      17        1   3.478769
8     -0.094513 -0.174976      18        1   2.599729
29     0.099604 -0.070551      19        1   1.471706
9      0.074700  0.001847      20        1   1.308322
6     -0.090618  0.000134      21        1   1.308322
24    -0.017649 -0.091551      22        1   1.308321
17    -0.012390  0.046962      23        1   1.086655
22     0.028227  0.052274      24        1   1.026549
3     -0.039276 -0.038478      25        1   1.026549
0      0.003938  0.002803      26        1   0.781068
2     -0.032043  0.004011      27        1   0.767210
21    -0.004947 -0.011487      28        1   0.582352
13    -0.004947 -0.011487      29        1   0.582352
15    -0.004947 -0.011487      30        1   0.582352, topic_info=       Term       Freq      Total Category  logprob  loglift
2     swiss  10.000000  10.000000  Default   30.000  30.0000
4       tip   6.000000   6.000000  Default   29.000  29.0000
26  careful   7.000000   7.000000  Default   28.000  28.0000
13      big  10.000000  10.000000  Default   27.000  27.0000
6      need  12.000000  12.000000  Default   26.000  26.0000
..      ...        ...        ...      ...      ...      ...
23      let   0.011508   3.191559  Topic30   -5.118  -0.4794
24  neutral   0.011508   2.629816  Topic30   -5.118  -0.2858
25    worry   0.011508   1.275637  Topic30   -5.118   0.4377
26  careful   0.011508   7.303637  Topic30   -5.118  -1.3073
28      man   0.011508   4.052034  Topic30   -5.118  -0.7181

[1582 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
124       8  0.773955  aesthetically
163      12  0.764489            age
11        6  0.783923           aged
95       18  0.818724          alert
81       10  0.765805          alone
...     ...       ...            ...
151       2  0.738092           wine
10        1  0.715335      wisconsin
25        6  0.783922          worry
103      17  0.782945          wound
162      11  0.780475          yodel

[223 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 26, 8, 24, 29, 2, 17, 6, 15, 27, 19, 12, 21, 13, 20, 11, 28, 9, 30, 10, 7, 25, 18, 23, 4, 1, 3, 22, 14, 16])